# Preprocessing

In [1]:
import io
import os
import sys
import PyPDF2
from pathlib2 import Path
from shutil import copyfile
import textract
import parallel_run
from words import splitted_words, replace_digits

Sposta i files che soddisfano un dato criterio da un albero di sottocartelle ad una cartella

In [2]:
def move_flattened_files(src_dir, out_dir, filt):
    p = Path(src_dir)
    if p.is_dir():
        for f in p.iterdir():
            if f.is_file() and filt(f):
                copyfile(str(f.absolute()),str(out_dir + u'/' + f.name))
            else:
                move_flattened_files(str(f.absolute()), out_dir, filt)

def file_filter(f):
    return '''f.name.endswith(u'001.pdf') and''' os.path.getsize(str(f.absolute()))<(1024**2)

In [ ]:
move_flattened_files('../raw_c','../atti_costitutivi2', file_filter)

In [5]:
move_flattened_files(u'../raw_a11',u'../atti_non_costitutivi2', file_filter)

Toglie i files con un certo criterio (dimensione maggiore al MB)

In [3]:
def delete_files(dir_name, pred):
    p = Path(dir_name)
    for f in p.iterdir():
        if (not f.is_dir()) and pred(str(f.absolute())):
            print str(f.absolute())
            os.remove(str(f.absolute()))

In [ ]:
delete_files('../atti_costitutivi', lambda f: os.path.getsize(f)>(1024**2))

Utils

In [7]:
orig = [f[:-4] for f in os.listdir("../atti_costitutivi")]
orig2 = [f[:-4] for f in os.listdir("../atti_non_costitutivi")]
f1 = [f[:-4] for f in os.listdir("../atti_costitutivi_txt")]
f2 = [f[:-4] for f in os.listdir("../ocr/scans_cost/")]

In [8]:
scans_filtered = [f for f in os.listdir("../ocr/scans_non_cost")]
#scans_filtered

Separa le scansioni di doc non costitutivi e le sposta in un'altra cartella

In [ ]:
for i in scans_filtered:
    os.rename('../ocr/scans/'+i+'.pdf', '../ocr/scans_non_cost/'+i+'.pdf')
    print i + '.pdf'

Converte i pdf in file di testo. sposta i fallimenti in altre cartelle.

In [4]:
def extract_text(f,out_dir, scans_dir, unhandled_dir, minlen):
    file = str(f.absolute())
    #print f.name
    try:
        text = textract.process(file).strip()
        if len(splitted_words(text)) <= minlen:
            print f.name, "is empty"
            copyfile(file, str(scans_dir + '/' + f.name))
        else:
            with open(out_dir + "/" + f.name[:-3] + 'txt', 'w') as outf:
                outf.write(text)
    except:
        print f.name
        copyfile(file, str(unhandled_dir + '/' + f.name))

def read_pdf_content(src_dir, out_dir, scans_dir, unhandled_dir, minlen):
    p = Path(src_dir)
    parallel_run.parallel_run_args(extract_text,((f, out_dir, scans_dir, unhandled_dir, minlen) for f in p.iterdir()))
    
    
def read_pdf_content_old(src_dir, out_dir, scans_dir, unhandled_dir, minlen):
    p = Path(src_dir)
    for f in p.iterdir():
        extract_text(f,out_dir, scans_dir, unhandled_dir, minlen)

In [51]:
read_pdf_content("../atti_costitutivi2", "../atti_costitutivi_txt", "../ocr/scans", "../unhadled_files", 100)

In [9]:
read_pdf_content('../atti_non_costitutivi2','../atti_non_costitutivi_txt2','../ocr/scans_non_cost2','../ocr/unhandled_files', 100)

3917207730001.pdf is empty
3667791020001.pdf is empty
3667352930001.pdf is empty
3833332710001.pdf is empty
3681162580001.pdf is empty
3490970240001.pdf is empty
3870964250001.pdf is empty
4044564540001.pdf is empty
3717123460001.pdf is empty
4034810400001.pdf is empty
3874632690001.pdf is empty
4121579530001.pdf
4112622480001.pdf is empty
3681329900001.pdf is empty
3313038160001.pdf is empty
3892235960001.pdf is empty
3802548140001.pdf is empty
3544782950001.pdf is empty
3718321110001.pdf is empty
3540267330001.pdf is empty
3509903500001.pdf is empty
3455840530001.pdf is empty
3978494680001.pdf is empty
3894168130001.pdf is empty
3523563810001.pdf is empty
3601509070001.pdf is empty
3504802500001.pdf is empty
3985547270001.pdf is empty
3995189810001.pdf is empty
3972996540001.pdf is empty
3772492790001.pdf is empty
3952315310001.pdf is empty
3826317790001.pdf is empty
3359633540001.pdf is empty
3880946330001.pdf is empty
3661467870001.pdf is empty
3355145290001.pdf is empty
3449506320

# GOOGLE VISION API 

Extract jpg's from pdf's. Quick and dirty.

In [2]:
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

import codecs

In [3]:
def convert_to_pdf(f, res):
    pdf = file("../ocr/scans_non_cost/" + f, "rb").read()

    startmark = "\xff\xd8"
    startfix = 0
    endmark = "\xff\xd9"
    endfix = 2
    i = 0

    njpg = 0
    while True:
        istream = pdf.find("stream", i)
        if istream < 0:
            break
        istart = pdf.find(startmark, istream, istream+20)
        if istart < 0:
            i = istream+20
            continue
        iend = pdf.find("endstream", istart)
        if iend < 0:
            return
            raise Exception("Didn't find end of stream!")
        iend = pdf.find(endmark, iend-20)
        if iend < 0:
            return
            raise Exception("Didn't find end of JPG!")

        istart += startfix
        iend += endfix
        #print "JPG %d from %d to %d" % (njpg, istart, iend)
        res.append(pdf[istart:iend])
        #jpgfile = file("ocr/tempjpg/" + f[:-4] + "_%d.jpg" % njpg, "wb")
        #jpgfile.write(jpg)
        #jpgfile.close()
        #print ("ocr/tempjpg/" + f[:-4] + "_%d.jpg" % njpg)

        #njpg += 1
        i = iend

In [4]:
# Instantiates a client
client = vision.ImageAnnotatorClient()

def convert_to_txt(img):
    image = types.Image(content=img)
    # Performs label detection on the image file
    response = client.text_detection(image=image)
    labels = response.text_annotations
    
    try:
        return (labels[0].description)
    except:
        return ""   

In [ ]:
for i,f in enumerate(os.listdir("../ocr/scans_non_cost")):
    print i
    images = []
    convert_to_pdf(f, images)
    if len(images) > 0:
        res = ""
        for el in images:
            txt = convert_to_txt(el)
            res += txt
        if res != "":        
            with codecs.open("../ocr/scanned_non_costitutivi/" + f[:-4] + ".txt", "wb", 'utf-8') as outf:        
                outf.write(u'' + res)
        else:
            print f + " is empty"

In [ ]:
# Conversione elemento singolo (per effettuare test)

# Instantiates a client
client = vision.ImageAnnotatorClient()

def convert_to_txt(image):
    # Loads the image into memory
    with io.open(image, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    # Performs label detection on the image file
    response = client.text_detection(image=image)
    labels = response.text_annotations

    print('Labels:')
    for label in labels:
        print(label.description)